In [1]:
# Importação de bibliotecas
import pandas as pd
from numpy import nan

from pendulum import now

### Tabela 1612 - Área plantada, área colhida, quantidade produzida, rendimento médio e valor da produção das lavouras temporárias

In [2]:
# Objetos
fonte = 'sidra'
dataset_name = 'tab1612_lavoura_temporaria'
variavel = 'area_colhida'

raiz = '/home/eduardo/repositorios/IS_Agro/databases'

ultimo_ano = now().year - 1

Área colhida (ha)

#### Bronze

In [3]:
# Lista de códigos de cultivos agrícolas da tabela 1612 do SIDRA 
culturas = [2688, 2689, 2690, 2691, 2692, 2693, 2694, 2695, 2696, 2697, 2698, 2699, 2700, 2701, 2702, 2703, 2704, 2705, 2706, 2707,
            2708, 2709, 2710, 2711, 2712, 2713, 2714, 2715, 2716, 109179, 109180]

contador = 1

In [ ]:
for ano in range(2003, ultimo_ano):
    for cultura in culturas:
        if contador == 1:
            df_1612 = pd.read_json(f'https://apisidra.ibge.gov.br/values/t/1612/n6/all/v/216/p/{ano}/c81/{cultura}')
            contador += 1
        else:
            df_1612 = pd.concat(
                [df_1612,
                 (pd.read_json(f'https://apisidra.ibge.gov.br/values/t/1612/n6/all/v/216/p/{ano}/c81/{cultura}')
                  .iloc[1:])],
                  ignore_index=True
                  )
            contador += 1

df_1612.to_json(f'{raiz}/bronze_{fonte}_{dataset_name}_{variavel}.json')

print(f'Foram contabilizadas {contador} repetições de dados de áreas de colheita raspados.')

#### Prata

| Código | Cultura                            |
|--------|------------------------------------|
| 2688   | Abacaxi*                           |
| 2689   | Algodão herbáceo (em caroço)       |
| 2690   | Alho                               |
| 2691   | Amendoim (em casca)                |
| 2692   | Arroz (em casca)                   |
| 2693   | Aveia (em grão)                    |
| 2694   | Batata-doce                        |
| 2695   | Batata-inglesa                     |
| 2696   | Cana-de-açúcar                     |
| 2697   | Cebola                             |
| 2698   | Centeio (em grão)                  |
| 2699   | Cevada (em grão)                   |
| 2700   | Ervilha (em grão)                  |
| 2701   | Fava (em grão)                     |
| 2702   | Feijão (em grão)                   |
| 2703   | Fumo (em folha)                    |
| 2704   | Juta (fibra)                       |
| 2705   | Linho (semente)                    |
| 2706   | Malva (fibra)                      |
| 2707   | Mamona (baga)                      |
| 2708   | Mandioca                           |
| 2709   | Melancia                           |
| 2710   | Melão                              |
| 2711   | Milho (em grão)                    |
| 2712   | Rami (fibra)                       |
| 2713   | Soja (em grão)                     |
| 2714   | Sorgo (em grão)                    |
| 2715   | Tomate                             |
| 2716   | Trigo (em grão)                    |
| 109179 | Girassol (em grão)                 |
| 109180 | Triticale (em grão)                |

In [3]:
# Tratamento dos dados
df_1612 = (
    pd.read_json(f'{raiz}/bronze_{fonte}_{dataset_name}_{variavel}.json')
          .drop(0, axis = 0) # Exclusão da linha relativa a descrição de cada coluna
          .filter(['D1C', 'D3C', 'D4C', 'V']) # Seleção das colunas a serem utilizadas
          .assign(**{
            'area': lambda df: pd.to_numeric(df['V'], errors='coerce'), # Coluna "area" como float a partir da coluna "V". Em hectares
            'geocodigo': lambda df: df['D1C'].astype(int), # Coluna de códigos do IBGE para cada município em inteiro, a partir da coluna "D1C"
            'data': lambda df: pd.to_datetime(df['D3C'], format='%Y'), # Coluna "data" em datetime baseado na ISO 8601, convertido da coluna "D3C"
            'cultura': lambda df: df['D4C'].astype(int) # Coluna de códigos relativos aos cultivos agrícolas em inteiro, a partir da coluna "D4C" 
            })
          .drop(['D1C', 'D3C', 'D4C', 'V'], axis='columns') # Exclusão das colunas originais
          .dropna(subset='area') # Limpeza de linhas com possíveis valores nulos encontrados apenas na coluna "area"
          .set_index(['geocodigo', 'data', 'cultura']) # Organização das novas colunas
)

df_1612.to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_{variavel}.parquet') # Exportação em .parquet

### Ouro

In [4]:
dataset_ouro = 'ouro_volume_agricola_lavouras'

In [5]:
# Junção das quatro tabelas de cultivos com dupla safra
dfs = ('df_839', 'df_1000', 'df_1001', 'df_1002') # Conjunto com nomes das tabelas a serem utilizados nos dataframes
tabelas = ('tab839_milho', 'tab1000_amendoim', 'tab1001_batata-inglesa', 'tab1002_feijao') # Conjunto com os nomes das tabelas encontradas nos nomes dos arquivos

# Criação de dataframe vazio para guardar as tabelas concatenadas
dfx = pd.DataFrame()

# Iteração de cada tabela 
for df, tabela in zip(dfs, tabelas):
    globals()[df] = (pd.read_parquet(f'{raiz}/prata_{fonte}_{tabela}_{variavel}.parquet')
                     .reset_index()
                     .assign(area=lambda x: x.area.replace({nan:0})) # Substituição de dado inválido por zero
                     )
    #Criação de dataframe distinto do primeiro para que os dados de fração da primeira safra (FPS) batam com respectivos municípios e anos
    globals()[df + '_fps'] = (globals()[df]
        .filter(['geocodigo', 'data', 'cultura'])
        .drop_duplicates()
        .reset_index(drop=True)
        .assign( # Cálculo da FPS de cada cultura a partir da condição de qual cultura é
            FPS=globals()[df].iloc[1::3, 4].reset_index(drop=True) / globals()[df].iloc[::3, 4].reset_index(drop=True) 
                    if (globals()[df].cultura.iloc[0] == 2711 or globals()[df].cultura.iloc[0] == 2691) else # Milho ou Amendoim
                    globals()[df].iloc[1::4, 4].reset_index(drop=True) / globals()[df].iloc[::4, 4].reset_index(drop=True)
        )
        .convert_dtypes()
    )
    # Concatenação dos dataframes
    dfx = pd.concat([dfx, globals()[df + '_fps']]).reset_index(drop=True)

In [6]:
# Leitura dos dados da base prata da tabela 1612 do SIDRA
df_1612 = pd.read_parquet(f'{raiz}/prata_{fonte}_{dataset_name}_{variavel}.parquet')

In [7]:
# Desenvolvimento de dataframe com dados da primeira safra
df_1612_ps = (df_1612.reset_index()
              .merge(dfx, # Junção do dataframe de lavouras temporárias com o de cultivos agrícolas de dupla safra 
                     how='left',
                     on=['geocodigo', 'data', 'cultura']
                     )
              .assign(FPS=lambda x: x.FPS.replace({nan:1})) # Substituição dos dados inválidos por 1 
              .assign(valor=lambda df: df.area * df.FPS) # Nova coluna resultado da multiplicação de cada valor da coluna "area" (dataframe da tabela 1612) com o de cada valor da coluna "FPS" (dataframe de dupla safra) 
              .dropna(subset='valor') # Descarte de linhas com possíveis valores nulos encontrados apenas na coluna "valor"
              .drop(['area','FPS'], axis='columns') # Exclusão das colunas agora desnecessárias
              .convert_dtypes() # Conversão de tipos de dados, para caso tenha algum como "object"
              .assign(
                  safra='1a. safra',
                  valor=lambda df: df.apply(lambda x:
                                            x.valor * 0.17 if (x.cultura == 2689 and str(x.geocodigo)[:2] == '51') else # Condição caso valores forem 'algodão herbáceo' em cultura e 'Mato Grosso' como UF
                                            x.valor * 0.79 if (x.cultura == 2689 and str(x.geocodigo)[:2] == '50') else # Condição caso valores forem 'algodão herbáceo' em cultura e 'Mato Grosso do Sul' como UF
                                            0 if (x.cultura == 2689 and str(x.geocodigo)[:2] not in ('17', '21', '22', '23', '24', '25', '26', '27', '28', '29', '50', '51')) else # Condição caso seja 'algodão herbáceo' em cultura e não sejam as UFs 'TO', estados do NO, 'MT' e 'MS'
                                            0 if (x.cultura in (2693, 2698, 2699, 2705, 2714, 2716, 109179, 109180)) else # Condição caso sejam os cultivos 'aveia', 'centeio', 'linho', 'sorgo', 'trigo', 'girassol' e 'triticale'
                                            x.valor, axis = 1) # Sem cálculo caso não se enquadre em nenhuma das condições anteriores
                                             )
              .reset_index(drop=True) # Criação de novo índice ordenado, descartando o anterior
            )

#df_1612_ps.to_parquet(f'{raiz}/{dataset_ouro}_primeira_safra.parquet', index=False)

In [10]:
# Desenvolvimento de dataframe com dados da segunda safra
df_1612_ss = (pd.merge(
                        df_1612.dropna(subset='area'), df_1612_ps, # Junção do dataframe original de lavouras temporárias com o de primeira safra 
                        how='left',
                        on=['geocodigo', 'data', 'cultura']
                      )
                      .assign(valor=lambda x: (x.area - x.valor).where(pd.notna(x.valor), x.area), # Nova coluna com dados relativos a segunda safra
                              safra='2a. safra')
                      .drop('area', axis='columns')
                )

#df_1612_ps.to_parquet(f'{raiz}/{dataset_ouro}_segunda_safra.parquet', index=False)

In [11]:
# Desenvolvimento de dataframe com dados organizados pelos cultivos agrícolas e suas safras
df_1612_final = (pd.concat([df_1612_ps, df_1612_ss], ignore_index=True)
                 .query('valor != 0') # Exclusão de dados iguais a zero por serem desprezíveis
                 .set_index(['geocodigo', 'data', 'cultura', 'safra'])
                 )
#df_1612_final.to_csv(f'{raiz}/{dataset_ouro}_temporarias.csv')

In [16]:
# Exportação para uso no indicador de Área Agropecuária
(
    df_1612_final
    .reset_index()
    .assign(geocodigo_uf = lambda df: df.geocodigo// 100000)
    .groupby(['geocodigo_uf', 'data', 'cultura', 'safra'])
    .agg({'valor':'sum'})
    .rename_axis(index={'geocodigo_uf': 'geocodigo'})
).to_csv(f'{raiz}/ouro_area_agropecuaria_lavouras_temporarias.csv')

In [10]:
# Desenvolvimento de dataframe com valores totais por municípios
df_1612_consolidado_munic = (df_1612_final.reset_index()
                             .groupby(['geocodigo', 'data', 'safra'])
                             .agg({'valor':sum})
                             )
df_1612_consolidado_munic.to_csv(f'{raiz}/{dataset_ouro}_temporarias_munic.csv')

In [11]:
# Desenvolvimento de dataframe com valores totais por Estados
df_1612_consolidado_uf = (df_1612_final.reset_index()
                          .assign(geocodigo_uf = lambda df: df.geocodigo// 100000)
                          .groupby(['geocodigo_uf', 'data', 'safra'])
                          .agg({'valor':sum})
                          .rename_axis(index={'geocodigo_uf': 'geocodigo'})
                          )
df_1612_consolidado_uf.to_csv(f'{raiz}/{dataset_ouro}_temporarias_uf.csv')